<a href="https://colab.research.google.com/github/iestleija/4808-Programacion/blob/Unidad1/Unidad1/Practices/Practica6_266229.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Maestría en Intelingencia Artificial y Analítica de Datos**

- Curso: Programación para Analítica Descriptiva y Predictiva

- Semestre: Enero-Junio

- Profesor: Dr. Vicente García Jiménez
- Práctica: 06 Manejo de librería Pandas
- Alumno: Ismael Estrada Leija
- Matrícula: 266229

# Práctica #6 - Unidad 1. Manejo de librería Pandas

## Preparación del ambiente
Instrucciones: Carga el archivo titanic.csv en la carpeta correspondiente de Google drive para realizar los siguientes ejercicios:

In [156]:
# Importación de la librería Pandas
import pandas as pd

# Leer el archivo CSV
# La separación por default es coma (,)
t_df = pd.read_csv('/content/drive/MyDrive/ClassFiles/Titanic-Dataset.csv')
#
t_df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S



## Ejercicio 1: Análisis de la distribución de supervivencia por combinación de sexo y clase del pasajero.

 - Calcula la proporción de supervivencia para cada combinación de 'Sex' y 'Pclass'.
  - Identifica qué combinación tuvo la tasa de supervivencia más alta.
 - Identifica qué combinación tuvo la tasa de supervivencia más baja.

### Adecuación de los datos en *t_df*
Para poder aplicar la función agregada de suma, requerimos segregar el dato *Survived* identificando cada pasajero como sobreviviente o fallecido.
Lo anterior lo logramos agregando las columnas *alive* y *dead*, calculadas a partir de *Survived*.

In [157]:
t_df['alive'] = t_df['Survived'].apply(lambda x: 1 if x == 1 else 0)
t_df['dead'] = t_df['Survived'].apply(lambda x: 1 if x == 0 else 0)

### Creacion del reporte agrupado por las combinaciones de *Sex* y *Pclass*
Se crea un reporte agregado según las combinaciones resultantes de *Sex* y *Pclass*, ejecutando la función agregada de suma para contar los casos encontrados en las columnas *alive* y *dead*.


In [158]:
survival_report = t_df.groupby(['Sex', 'Pclass']).agg(AliveCount = ('alive','sum'), DeadCount = ('dead','sum'))
print(survival_report)

               AliveCount  DeadCount
Sex    Pclass                       
female 1               91          3
       2               70          6
       3               72         72
male   1               45         77
       2               17         91
       3               47        300


### Cálculo de la Razón de Supervivencia
Se agrega la columna SurvivalRate para calcular para cada línea del reporte agrupado la razón de supervivencia. Dicha razón se calcula aplicando la fórmula siguiente:
$$SurvivalRate = \frac {Alive Count}{Total Count}$$

In [159]:
survival_report['SurvivalRate'] = survival_report['AliveCount'] / (survival_report['AliveCount'] + survival_report['DeadCount'])
print(survival_report)

               AliveCount  DeadCount  SurvivalRate
Sex    Pclass                                     
female 1               91          3      0.968085
       2               70          6      0.921053
       3               72         72      0.500000
male   1               45         77      0.368852
       2               17         91      0.157407
       3               47        300      0.135447


### Identificación de la tasa de supervivencia mas alta

In [160]:
max_survival = survival_report.loc[survival_report['SurvivalRate'] == survival_report['SurvivalRate'].max()]
print("La combinación con mayor razón de supervivencia fue:\n")
print(max_survival)

La combinación con mayor razón de supervivencia fue:

               AliveCount  DeadCount  SurvivalRate
Sex    Pclass                                     
female 1               91          3      0.968085


### Identificación de la tasa de supervivencia mas baja

In [161]:
min_survival = survival_report.loc[survival_report['SurvivalRate'] == survival_report['SurvivalRate'].min()]
print("La combinación con menor razón de supervivencia fue:\n")
print(min_survival)

La combinación con menor razón de supervivencia fue:

             AliveCount  DeadCount  SurvivalRate
Sex  Pclass                                     
male 3               47        300      0.135447


## Ejercicio 2: Identificación de familias grandes a bordo.

 - Crea una nueva columna 'FamilySize' sumando las columnas 'SibSp' y 'Parch'.
 - Considera como "familia grande" a aquellas donde 'FamilySize' es mayor a 3.
 - Calcula el número de pasajeros en familias grandes.
 - Calcula la proporción de supervivencia entre los pasajeros que pertenecen a familias grandes.

### Adición de columnas conforme a lo requerido.
 - Se adiciona la columna Family Size como se pidió
 - Se adiciona la columna BigFamily como bandera lógica para indicar que familias son consideradas grandes.

In [162]:
t_df['FamilySize'] = t_df['SibSp'] + t_df['Parch']
t_df['BigFamily'] = t_df['FamilySize'] > 3

### Conteo de pasajeros en familias grandes.


In [163]:
pasajeros_familias_grandes = t_df[t_df['BigFamily']]['PassengerId'].count()
print(f"La cantidad de pasajeros en familias grandes fue:{pasajeros_familias_grandes}")

La cantidad de pasajeros en familias grandes fue:62


### Cálculo de supervivencia para pasajeros en familias grandes
Reutilizando la fórmula de supervivencia mencionada en el ejercicio anterior se presenta enseguida la razón de supervivencia para los pasajeros pertenecientes a familias grandes.

In [164]:
big_family_survival_rate = t_df[t_df['BigFamily']]['alive'].count() / (t_df[t_df['BigFamily']]['alive'].count() + t_df[t_df['BigFamily']]['dead'].count())
print(f"La proporción de supervivencia para pasajeros en familias grandes fue: {big_family_survival_rate}")

La proporción de supervivencia para pasajeros en familias grandes fue: 0.5


## Ejercicio 3: Segmentación por grupos de edad.

Clasifica a los pasajeros en las siguientes categorías de edad(tip puede resultar mas sencillo realizarlo con una función):  menor de edad (< 18) y mayor de edad (>=18)

In [165]:
def age_category(age):
    if age < 18:
        return 'Minor'
    else:
        return 'Adult'

#t_df['AgeCategory'] = t_df['Age'].apply(age_category)
t_df['AgeCategory'] = t_df['Age'].apply(age_category)
t_df.groupby(t_df['AgeCategory'])['PassengerId'].count()


,PassengerId
AgeCategory,
Adult,778
Minor,113


## Ejercicio 4: Comparación entre promedios calculados manualmente y con Pandas

 - Utiliza NumPy para calcular el promedio de las columnas 'Age' y 'Fare', ignorando valores nulos.
 - Compara estos valores con los promedios obtenidos utilizando los métodos nativos de Pandas.
 - Verifica que los resultados sean consistentes.

### Promedios calculados para las columnas requeridas


**Promedios de edades**

In [166]:
import numpy as np
import tabulate as tabu

# Calculate Age average using Numpy
np_age_average = np.mean(t_df['Age'])
np_age_average_no_nulls = np.nanmean(t_df['Age'])
pd_age_average = t_df['Age'].mean()


print(f"The average ages are:")
print(f"{'- by Numpy, regular mean:':<42} {np_age_average:>10}")
print(f"{'- by Numpy, nanmean ignoring nan values:':<42} {np_age_average_no_nulls:>10}")
print(f"{'- by Pandas mean method:':<42} {pd_age_average:>10}")

The average ages are:
- by Numpy, regular mean:                  29.69911764705882
- by Numpy, nanmean ignoring nan values:   29.69911764705882
- by Pandas mean method:                   29.69911764705882


**Promedios de tarifas**

In [167]:
# Calculate Fare average using Numpy
np_fare_average = np.mean(t_df['Fare'])
np_fare_average_no_nulls = np.nanmean(t_df['Fare'])
pd_fare_average = t_df['Fare'].aggregate('mean')


print(f"The average fares are:")
print(f"{'- by Numpy, regular mean method:':<42} {np_fare_average:>10}")
print(f"{'- by Numpy, nanmean ignoring nan values:':<42} {np_fare_average_no_nulls:>10}")
print(f"{'- by Pandas aggregate mean method:':<42} {pd_fare_average:>10}")


The average fares are:
- by Numpy, regular mean method:           32.204207968574636
- by Numpy, nanmean ignoring nan values:   32.204207968574636
- by Pandas aggregate mean method:         32.204207968574636


## Ejercicio 5. Creación de intervalos de clase usando NumPy y análisis con Pandas

 - Divide la columna 'Fare' en 5 intervalos equidistantes utilizando la función numpy.linspace, el estudiante deberá investigar la operación de esta función en python.
 - Crea una nueva columna en el DataFrame que asigne a cada pasajero el intervalo correspondiente de su tarifa.
 - Calcula el número de pasajeros en cada intervalo utilizando Pandas y la proporción de supervivientes por intervalo.

### Creación de los límites de intervalo usando Numpy.linspace
Una vez investigada la función numpy.linspace (Mohit, 2025), se procede a crear los límites de los 5 intervalos estadísticos dependiendo de los datos existentes en la columna *Fare* del dataframe.


In [168]:
range_limits = np.linspace(t_df['Fare'].min(), t_df['Fare'].max(), num=6)
print(f"los límites de rango encontrados son: {range_limits}")

los límites de rango encontrados son: [  0.      102.46584 204.93168 307.39752 409.86336 512.3292 ]


### Aplicación de los intervalos en nueva columna con Pandas.cut
Para la aplicación de los intervalos en una nueva columna se cuenta con la función de pandas.cut; se ejecuta con ella la asignación de intervalos y se comprueba que los intervalos son iguales con la única diferencia del ajuste automático para incluír el límite inferior del primer intervalo.

In [169]:
t_df['fare_interval'] = pd.cut(t_df['Fare'], 5)
intervals = t_df['fare_interval'].unique()
print(f"Los intervalos son: {intervals}")


Los intervalos son: [(-0.512, 102.466], (204.932, 307.398], (102.466, 204.932], (409.863, 512.329]]
Categories (5, interval[float64, right]): [(-0.512, 102.466] < (102.466, 204.932] <
                                           (204.932, 307.398] < (307.398, 409.863] <
                                           (409.863, 512.329]]


### Cálculo del número de pasajeros por intervalo

In [170]:
pass_by_interval=t_df.groupby('fare_interval',observed=False).agg(Pasajeros_por_Intervalo = ('PassengerId','count'))
print(pass_by_interval)

                    Pasajeros_por_Intervalo
fare_interval                              
(-0.512, 102.466]                       838
(102.466, 204.932]                       33
(204.932, 307.398]                       17
(307.398, 409.863]                        0
(409.863, 512.329]                        3


### Calculo de pasajeros y supervivencia por intervalo de tarifa
Habiendo encontrado la forma de aplicar una función durante el proceso de agrupamiento, se calculan en un solo paso, por cada intervalo, el total de pasajeros y la razón de supervivencia.


In [171]:
def interval_aggregation(group):
    survival_rate =  group['alive'].sum() / group['PassengerId'].count()
    total_passengers = group['PassengerId'].count()
    return pd.Series({
        'Total de Pasajeros': total_passengers,
        'Razón de supervivencia': survival_rate})

fare_interval_report=t_df.groupby('fare_interval',observed=False).apply(interval_aggregation,include_groups=False)
print(fare_interval_report)

                    Total de Pasajeros  Razón de supervivencia
fare_interval                                                 
(-0.512, 102.466]                838.0                0.361575
(102.466, 204.932]                33.0                0.757576
(204.932, 307.398]                17.0                0.647059
(307.398, 409.863]                 0.0                     NaN
(409.863, 512.329]                 3.0                1.000000


/tmp/ipython-input-2861005734.py:2: RuntimeWarning: invalid value encountered in scalar divide
  survival_rate =  group['alive'].sum() / group['PassengerId'].count()


## Conclusión
Durante la presente práctica se probaron diversas funcionalidade de las librerías Pandas y Numpy con la ayuda del material académico asi como de algunos recursos en línea como (Kumar, 2025) y (Mohit, 2025)

# Referencias
Kumar, A. (2025). Pandas Tutorial. GeeksforGeeks. https://www.geeksforgeeks.org/pandas/pandas-tutorial/

Mohit, G. (2025). NumPy linspace() Method | Create Evenly Spaced Array. GeeksforGeeks. https://www.geeksforgeeks.org/python/numpy-linspace/


⚠ No olvides enviar el archivo TXT con la liga a tu notebook en Google Colab y Github